# Creating the Reviews Dataset

In this notebook, will be generated the dataset of Steams Reviews. In this example, we will use the language `Brazilian Portuguese` to generate the data, but you can use another (or all languages) by passing the `language` argument in the `download_reviews` function. In this [page](https://partner.steamgames.com/doc/store/localization) you can see every language available.

## Importing some packages, libs and modules

In [1]:
import json, csv, copy, os, requests

## Getting all the apps from the Steam

In [20]:
apps_url = "http://api.steampowered.com/ISteamApps/GetAppList/v2"
apps_data = requests.get(url=apps_url, params={'json': 1}).json()

In [17]:
def download_batch(response, params):
    
    # Check if the request don't returned a json
    if response is None:
        raise ConnectionRefusedError('The request returned null.')

    # Check if the request was not successfull.
    if 'success' not in response:
        raise ConnectionError('The request was not a success.')
    
    # results from this function
    result = {}
    
    # If the query return at least one review
    if response['query_summary']['num_reviews'] > 0:
        result['reviews'] = response['reviews']
        result['reviews_count'] = response['query_summary']['num_reviews']
        result['finished'] = False
        
        params['cursor'] = response['cursor']
    
    # No more reviews left
    else:
        result['finished'] = True
    
    # add the updated params
    result['params'] = params
    
    # return the information
    return result

def download_reviews(appid, language='brazilian'):
    
    # Information to request the reviews from the game
    steam_url = 'https://store.steampowered.com/appreviews/'
    params = {
        'json': 1,
        'filter': 'recent', # sort by: recent, update
        'language': language, # only reviews in this language
        'cursor': '*',
        'review_type': 'all', # positive and negative reviews
        'purchase_type': 'all', # all, non_steam_purchase, steam
        'num_per_page': '100', # 100 is the max number
    }
    
    # response from the request in format json
    response = requests.get(url=steam_url+appid, 
                            params=params).json()
    
    # If there is no total_reviews, the query returns
    # zero reviews from this game
    if 'total_reviews' not in response['query_summary']:
        return {}
    
    # Dict with the info to be returned
    reviews = {
        'reviews': [],
        'total_reviews': response['query_summary']['total_reviews'],
        
    }
    
    # request all the reviews from this app_id
    reviews_count = 0
    while True:
        
        # download a batch of 100 reviews
        result = download_batch(response, params)
        
        # If all the reviews was downloaded, break the loop
        if result['finished']:
            break
        
        # update with the results from the batch
        params = result['params']
        reviews['reviews'] += result['reviews']
        reviews_count += result['reviews_count']
                
        # request with the next reviews
        response = requests.get(url=steam_url+appid, 
                                params=params).json()

    # return all the reviews downloaded
    return reviews